In [2]:
from imp import reload
import utils as ul
import mlr as pb
import fw
import numpy as np
#reload(pb)

## Data generation

In [3]:
## X: n*p  W: p*m  Y: n*m
np.random.seed(0)
n = 16; m = 5; p = 4
X, y, W = pb.generate(n = 16)

In [4]:
y

array([3, 0, 2, 3, 0, 2, 0, 3, 4, 3, 2, 2, 2, 4, 1, 4])

In [5]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(X,y).score(X,y)

0.9375

In [6]:
points = ul.mat2point(X,y)
dataRDD = sc.parallelize(points).mapPartitions(ul.point2mat)

In [8]:
dataRDD.first()

[array([[ 1.76405235,  0.40015721,  0.97873798,  2.2408932 ],
        [ 1.86755799, -0.97727788,  0.95008842, -0.15135721]]), array([[3],
        [0]])]

In [9]:
dataRDD = dataRDD.map(lambda s: (s[0], np.reshape(s[1],len(s[1])), m))

In [10]:
dataRDD.first()

(array([[ 1.76405235,  0.40015721,  0.97873798,  2.2408932 ],
        [ 1.86755799, -0.97727788,  0.95008842, -0.15135721]]),
 array([3, 0]),
 5)

In [11]:
hashedDataRDD = dataRDD.map(ul.hashkey)
hashedDataRDD

PythonRDD[4] at RDD at PythonRDD.scala:43

In [12]:
hashedDataRDD.first()

(2364037124536233116,
 (array([[ 1.76405235,  0.40015721,  0.97873798,  2.2408932 ],
         [ 1.86755799, -0.97727788,  0.95008842, -0.15135721]]),
  array([3, 0]),
  5))

In [13]:
modelRDD = hashedDataRDD.mapValues(pb.Model).persist()
modelRDD

PythonRDD[6] at RDD at PythonRDD.scala:43

In [14]:
modelRDD.collect()

[(2364037124536233116, <mlr.Model at 0x7feeda054a90>),
 (-598135049527906511, <mlr.Model at 0x7feeda054c50>),
 (5383232059404804407, <mlr.Model at 0x7feeda054b00>),
 (786678581264638161, <mlr.Model at 0x7feeda0540b8>),
 (-4789202252097188837, <mlr.Model at 0x7feeda054160>),
 (6020462086823944251, <mlr.Model at 0x7feeda054be0>),
 (-7092696113887687326, <mlr.Model at 0x7feeda054a58>),
 (-271061975423292267, <mlr.Model at 0x7feeda054c18>)]

## Initialization

In [15]:
np.random.seed(0)

T = 30               # number of iterations of FW
U = np.zeros((T,p))  # left singular vector
V = np.zeros((T,m))  # right singular vector

loss = np.zeros(T)

paramRDD = modelRDD.mapValues(lambda x: pb.Param((x.p, x.m)))
paramRDD

PythonRDD[7] at RDD at PythonRDD.scala:43

In [16]:
paramRDD.collect()

[(2364037124536233116, <mlr.Param at 0x7feeda054630>),
 (-598135049527906511, <mlr.Param at 0x7feeda0549b0>),
 (5383232059404804407, <mlr.Param at 0x7feeda0544e0>),
 (786678581264638161, <mlr.Param at 0x7feeda054b70>),
 (-4789202252097188837, <mlr.Param at 0x7feeda054ba8>),
 (6020462086823944251, <mlr.Param at 0x7feeda054978>),
 (-7092696113887687326, <mlr.Param at 0x7feeda0543c8>),
 (-271061975423292267, <mlr.Param at 0x7feeda054748>)]

In [17]:
paramRDD.first()[1].W

array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

## Subfunctions

In [18]:
gradRDD = fw.gradient(paramRDD, modelRDD)
gradRDD

PythonRDD[16] at RDD at PythonRDD.scala:43

In [19]:
gradRDD.first()

(-598135049527906511,
 array([[ 0.13156377,  0.13156377,  0.23478263, -0.62947395,  0.13156377],
        [ 0.1064547 ,  0.1064547 , -0.3041438 , -0.01522031,  0.1064547 ],
        [ 0.11758136,  0.11758136, -0.02646221, -0.32628187,  0.11758136],
        [ 0.35758957,  0.35758957, -1.09668394,  0.02391524,  0.35758957]]))

In [20]:
gradRDD.values().reduce(fw.add)

array([[-5.01405051,  0.55082395,  6.44565284, -4.67081879,  2.68839251],
       [ 1.90313056, -1.03614316,  0.27294416, -3.92724211,  2.78731055],
       [-1.74301036,  0.20022637,  2.41195662, -1.62431755,  0.75514493],
       [ 1.94878166,  1.11888603, -3.07113187, -1.89428243,  1.89774662]])

In [21]:
fw.centralize(gradRDD, 1)

(array([ 1.        ,  0.20060436,  0.354374  , -0.1545394 ]),
 array([-0.70897714,  0.03078158,  1.        , -0.73329464,  0.41149019]))

In [22]:
fw.warmstart(gradRDD).first()

(-598135049527906511,
 (array([-0.0935916 ,  0.28154771,  0.08720898,  1.        ]),
  array([ 0.31949585,  0.31949585, -1.        ,  0.04151244,  0.31949585])))

In [23]:
fw.warmstart(gradRDD).values().collect()

[(array([-0.0935916 ,  0.28154771,  0.08720898,  1.        ]),
  array([ 0.31949585,  0.31949585, -1.        ,  0.04151244,  0.31949585])),
 (array([ 1.        , -0.90532406,  0.00677307, -0.13692165]),
  array([-1.        ,  0.26327177,  0.26327177,  0.21018469,  0.26327177])),
 (array([ 0.39181225,  0.21241041,  0.46597718,  1.        ]),
  array([-0.32707349, -0.01877953, -0.32707349, -0.32707349,  1.        ])),
 (array([ 1.        , -0.22277663, -0.18509004,  0.04919791]),
  array([-0.66468565, -0.11177145,  1.        , -0.11177145, -0.11177145])),
 (array([ 0.64676459,  1.        ,  0.00374751, -0.13849697]),
  array([-0.0436782 , -0.0436782 , -0.0436782 , -0.86896541,  1.        ])),
 (array([ 1.        ,  0.06239272,  0.54471948,  0.97406005]),
  array([-0.26521571,  0.42173857,  0.42173857, -1.        ,  0.42173857])),
 (array([ 0.5265869 ,  0.6279321 ,  1.        , -0.92200247]),
  array([-0.25, -0.25,  1.  , -0.25, -0.25])),
 (array([ 1.        ,  0.25609078,  0.52635609, -0

In [24]:
fw.avgmix(gradRDD, 1)

(array([ 5.47157214,  1.31227294,  2.44969226,  1.5519613 ]),
 array([-2.62817261,  0.18326171,  2.31425868, -2.70312856,  2.83378077]))

In [25]:
fw.warmstart(gradRDD).map(lambda x: x[1][1]).collect()

[array([ 0.31949585,  0.31949585, -1.        ,  0.04151244,  0.31949585]),
 array([-1.        ,  0.26246877,  0.26246877,  0.21259368,  0.26246877]),
 array([-0.32707349, -0.01877953, -0.32707349, -0.32707349,  1.        ]),
 array([-0.66468565, -0.11177145,  1.        , -0.11177145, -0.11177145]),
 array([-0.0436782 , -0.0436782 , -0.0436782 , -0.86896541,  1.        ]),
 array([-0.2652158,  0.4217386,  0.4217386, -1.       ,  0.4217386]),
 array([-0.25, -0.25,  1.  , -0.25, -0.25]),
 array([-0.39701532, -0.39701532,  1.        , -0.39701532,  0.19104597])]

In [26]:
fw.warmstart(gradRDD).map(lambda x: x[1][1]).reduce(lambda x,y: x+y)

array([-2.62817257,  0.18245876,  2.31345569, -2.70071954,  2.83297767])

In [28]:
fw.poweriter(gradRDD, 0, lambda t: 10)

(array([ 0.91681161,  0.18392373,  0.32489397, -0.14167726]),
 array([-0.47683779,  0.02070286,  0.67257329, -0.49319938,  0.27676101]))

In [29]:
np.array([fw.loground(i) for i in range(101)])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3])

In [30]:
u, v = fw.centralize(gradRDD)
u, v

(array([ 1.        ,  0.20060436,  0.354374  , -0.15453941]),
 array([-0.70897714,  0.03078158,  1.        , -0.73329464,  0.41149019]))

In [31]:
u, v = fw.regularize(u, v, 1)
u, v

(array([-0.91681196, -0.18391648, -0.32489432,  0.14168358]),
 array([-0.47684045,  0.02070293,  0.67257522, -0.4931958 ,  0.27675811]))

In [32]:
paramRDD = fw.broadcast(paramRDD, u, v)
paramRDD.values().first().D

array([[ 0.43717303, -0.01898069, -0.616625  ,  0.45216781, -0.25373514],
       [ 0.08769882, -0.00380761, -0.12369767,  0.09070684, -0.05090038],
       [ 0.15492276, -0.00672626, -0.21851587,  0.16023652, -0.08991714],
       [-0.06756046,  0.00293326,  0.09529286, -0.06987774,  0.03921208]])

In [33]:
np.outer(-u,v)

array([[-0.43717303,  0.01898069,  0.616625  , -0.45216781,  0.25373514],
       [-0.08769882,  0.00380761,  0.12369767, -0.09070684,  0.05090038],
       [-0.15492276,  0.00672626,  0.21851587, -0.16023652,  0.08991714],
       [ 0.06756046, -0.00293326, -0.09529286,  0.06987774, -0.03921208]])

In [34]:
fw.loss(paramRDD, modelRDD)

25.751006598945605

In [35]:
foo = modelRDD.join(paramRDD).values().first()
foo[0].miss(foo[1].W)

2

In [36]:
fw.miss(paramRDD, modelRDD)

13

In [37]:
paramRDD, _ = fw.naivestep(paramRDD)
paramRDD.values().first().a

1.0

In [38]:
paramRDD = fw.descent(paramRDD)
paramRDD.values().first().W

array([[ 0.43717303, -0.01898069, -0.616625  ,  0.45216781, -0.25373514],
       [ 0.08769882, -0.00380761, -0.12369767,  0.09070684, -0.05090038],
       [ 0.15492276, -0.00672626, -0.21851587,  0.16023652, -0.08991714],
       [-0.06756046,  0.00293326,  0.09529286, -0.06987774,  0.03921208]])

In [39]:
fw.loss(paramRDD, modelRDD)

18.099751803048477

In [40]:
fw.miss(paramRDD, modelRDD)

7

In [41]:
paramRDD = fw.ascent(paramRDD)
paramRDD.values().first().W

array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [42]:
fw.loss(paramRDD, modelRDD)

25.751006598945605

## Main function

In [43]:
np.random.seed(0)
paramRDD = modelRDD.mapValues(lambda x: pb.Param((x.p, x.m)))
fw.iterate(modelRDD, paramRDD, fw.centralize, fw.naivestep, 1, 0)

(PythonRDD[353] at RDD at PythonRDD.scala:43,
 (array([-0.91681196, -0.18391648, -0.32489432,  0.14168357]),
  array([-0.47684045,  0.02070293,  0.67257522, -0.4931958 ,  0.27675811])),
 None,
 18.099751803138894,
 7)

In [44]:
np.random.seed(0)
paramRDD = modelRDD.mapValues(lambda x: pb.Param((x.p, x.m)))
fw.iterate(modelRDD, paramRDD, fw.avgmix, fw.naivestep, 1, 0)

(PythonRDD[378] at RDD at PythonRDD.scala:43,
 (array([-0.86438637, -0.20727636, -0.38699506, -0.24516839]),
  array([-0.49996465,  0.03485664,  0.44024217, -0.51420645,  0.53907229])),
 None,
 19.027746322078624,
 9)

In [45]:
np.random.seed(0)
paramRDD = modelRDD.mapValues(lambda x: pb.Param((x.p, x.m)))
fw.iterate(modelRDD, paramRDD, lambda x,t: fw.poweriter(x,t,lambda t: 1), fw.naivestep, 1, 0)

(PythonRDD[405] at RDD at PythonRDD.scala:43,
 (array([-0.91162982, -0.26262144, -0.31485946,  0.02871527]),
  array([-0.44092508,  0.02460663,  0.64078078, -0.54187515,  0.31741282])),
 None,
 18.201039029787516,
 8)

In [46]:
np.random.seed(0)
paramRDD = modelRDD.mapValues(lambda x: pb.Param((x.p, x.m)))
fw.iterate(modelRDD, paramRDD, lambda x,t: fw.poweriter(x,t,lambda t: 20), fw.naivestep, 1, 0)

(PythonRDD[470] at RDD at PythonRDD.scala:43,
 (array([-0.91681196, -0.18391648, -0.32489432,  0.14168357]),
  array([-0.47684045,  0.02070293,  0.67257522, -0.4931958 ,  0.27675811])),
 None,
 18.099751803252747,
 7)

In [47]:
np.random.seed(0)
paramRDD = modelRDD.mapValues(lambda x: pb.Param((x.p, x.m)))
fw.iterate(modelRDD, paramRDD, lambda x,t: fw.poweriter(x,t,fw.loground), fw.naivestep, 1, 0)

(PythonRDD[497] at RDD at PythonRDD.scala:43,
 (array([-0.91162981, -0.26262151, -0.31485945,  0.02871519]),
  array([-0.44092505,  0.02460663,  0.64078076, -0.54187519,  0.31741285])),
 None,
 18.201039131452792,
 8)

In [48]:
np.random.seed(0)
v0 = np.random.randn(m)
np.random.seed(0)
paramRDD = modelRDD.mapValues(lambda x: pb.Param((x.p, x.m)))
fw.iterate(modelRDD, paramRDD, lambda x,t: fw.poweriter(x,t,fw.loground,v0), fw.naivestep, 1, 0)

(PythonRDD[523] at RDD at PythonRDD.scala:43,
 (array([ 0.93695611,  0.04651203,  0.34566378, -0.0215969 ]),
  array([ 0.50042843, -0.04885705, -0.66216079,  0.48381387, -0.27322445])),
 None,
 18.259712499964124,
 8)